# Simulating the Kibble-Zurek Mechanism on a Quantum Computer

This notebook demonstrates the use of a quantum computer for ... . It has the following sections:

1. [The Kibble-Zurek Mechanism](#The-Kibble-Zurek-Mechanism) describes.
2. 

## The Kibble-Zurek Mechanism

The [[1]](#1) Nature paper

<img src="_static/temp1_kz_overview.png" width="500">

<img src="_static/tmp_coherent.png" width="800">

## References

<a name="1">[1]</a> King, A.D., Suzuki, S., Raymond, J. et al. 
Coherent quantum annealing in a programmable 2,000 qubit Ising chain. 
Nat. Phys. 18, 1324–1328 (2022). https://doi.org/10.1038/s41567-022-01741-6


In [ ]:
from dwave.system import DWaveSampler

# TO BE UPDATED: 
#sampler = DWaveSampler(solver={'topology__type': 'zephyr'})
sampler = DWaveSampler(solver="Advantage2_prototype2_...")
print(sampler.solver.name)

In [ ]:
import minorminer
from helpers.qa import create_bqm, find_one_to_one_embedding

ising_chain_length = 500

embedding = find_one_to_one_embedding(ising_chain_length, sampler.edgelist)

In [ ]:
from dwave.system import FixedEmbeddingComposite
# For now define a `placeholder_submission_if` until a formal API exists for such short anneal times


In [ ]:
times1 = [t for t in range(5, 10, 1)]       # First log section
times2 = [t for t in range(10, 100, 10)]    # second
times3 = [t for t in range(100, 1000, 200)] # third
anneal_times = times1 + times2 +times3

Js = [0.12, -0.7, -1.4]

qpu_params = {'sampler': sampler, 'embedding': embedding, 'num_reads': 5000}
sampleset = {}
for J in Js:    

    bqm = create_bqm(ising_chain_length, J)
    
    for ta in anneal_times:

        qpu_params['bqm'] = bqm
        qpu_params['label'] = f'example - Kibble-Zurek - J: {J} & t_a: {ta} ns'
        qpu_params['anneal_time'] = 0.001*ta

        sampleset[(J, ta)] = placeholder_submission_if(qpu_params)

In [ ]:
from helpers.kb_calcs import avg_kink_density

kink_densities = {}
for key, samples in sampleset.items():
    kink_densities[key] = avg_kink_density(samples, key[0])

In [ ]:
from helpers.plots import plot_kink_densities

plot_kink_densities(kink_densities, Js)

TEST FIELD

In [ ]:
import pandas as pd
import numpy as np

schedule = pd.read_csv('helpers/09-1302A-B_Advantage2_prototype2.2_annealing_schedule.csv')

A = schedule['A(s) (GHz)']
B1 = schedule['B(s) (GHz)']         # B for J=1 ? 
C = schedule['C (normalized)']

def theoretical_kink_density(annealing_times_ns, J):
    """
    Calculate the coherent-theory kink density for the coupling strength & annealing times. 

    Args:
        annealing_times_ns: iterable of annealing times in nanoseconds

        J: Coupling strength

    Returns:
        n(ta).  
    """
    B = B1*J
    A_tag = A.diff()/C.diff()
    B_tag = B.diff()/C.diff()

    sc_indx = abs(A - B).idxmin()

    b_top = (A[sc_indx]*1e9)*2*np.pi
    b_bottom = (B_tag[sc_indx]/B[sc_indx]) - (A_tag[sc_indx]/A[sc_indx])
    b = b_top/b_bottom

    return np.power([t*1e-9 for t in annealing_times_ns], -0.5)/(2*np.pi*np.sqrt(2*b))

print(theoretical_kink_density([5], 1))
print(theoretical_kink_density([5], -1))
print(theoretical_kink_density([5], 0.01))
print(theoretical_kink_density([5], -0.01))


Copyright &copy; 2024 D-Wave Systems, Inc

The software is licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a><br />This Jupyter Notebook is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>